In [47]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.utils import class_weight
import trimesh
import pymeshlab as pmlab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
num_vertices = 100

In [3]:
config = {
    'experiment_name': 'vertix_edge_overfitting',
    'device': 'cpu',  
    'is_overfit': True,
    'batch_size': 8,
    'resume_ckpt': False,
    'learning_rate': 5e-4,
    'max_epochs': 50000,
    'print_every_n': 10,
    'validate_every_n': 50,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'overfit',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2,
}

In [4]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "val", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [5]:
dataset.filter_data()

Length of dataset: 32304
Filtering data ..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32304/32304 [00:06<00:00, 5307.07it/s]

Length of dataset: 5474


In [82]:
from inference.inference_vertix_edge import InferenceHandlerVertixEdgeModel

inferer = InferenceHandlerVertixEdgeModel('runs/vertix_edge_hungarian_train/model_best.ckpt', config["num_vertices"], config["feature_size"], config["device"])

In [83]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [95]:
def visualize_random():
    random_sample = random.randint(0,len(dataset)-1)
    _, input_sdf, target_vertices, mask, target_edges, edges_adj = dataset[random_sample]
    faces = []

    for i in range(num_vertices):
        for j in range(i+1,num_vertices):
            for k in range(j+1,num_vertices):
                if i != j and j != k and k != i:
                    if target_edges[i][j] ==1 and target_edges[j][k]==1 and target_edges[k][i]==1:
                        faces.append(np.array([i,j,k]).reshape(1,-1))
                        
    faces = np.concatenate(faces,axis=0)
                        
                    
    print(trimesh.graph.face_adjacency(faces).shape)
    
            
      
    print("GT Mesh")
    
    
    visualize_mesh(target_vertices, faces)
    

    
    output_pointcloud, edges = inferer.infer_single(input_sdf, mask, x_indices,y_indices,edges_adj)
    
    faces = []
    
    
    print("Predicted Pointcloud")
    
    visualize_pointcloud(output_pointcloud,point_size=0.05)

    
    target_size = int(sum(mask))
    
    matched_edges = np.zeros((num_vertices,num_vertices))
                    
    cost = distance_matrix(output_pointcloud, target_vertices[:int(sum(mask))])
                        
    vertix_idx, target_idx = linear_sum_assignment(cost) 
    
    for i in range(target_size):
        for j in range(target_size):
            curr_v_1 = vertix_idx[i]
            curr_t_1 = target_idx[i]
            curr_v_2 = vertix_idx[j]
            curr_t_2 = target_idx[j]
            matched_edges[curr_v_1,curr_v_2] = target_edges[curr_t_1,curr_t_2]
            
    faces = []
    
    contrib = {}
    
    for v in range(100):
        contrib[v] = 0

    for i in range(num_vertices):
        for j in range(i+1,num_vertices):
            for k in range(j+1,num_vertices):
      
                if edges[i][j] and edges[j][k] and edges[k][i] and contrib[i] < 5 and contrib[j] < 5 and contrib[k] < 5:
                    contrib[i] += 1
                    contrib[j] += 1
                    contrib[k] += 1
                    faces.append(np.array([i,j,k]).reshape(1,-1))
                        
    faces = np.concatenate(faces,0)
    
        
    print(faces.shape)
    
    adj = trimesh.graph.face_adjacency(faces)
    
    print(adj.shape)
    

    #export_mesh_to_obj("output.obj",output_pointcloud.copy(),faces.copy())

    #print("Predicted Mesh")
        
    print(len(faces))

    
        
    visualize_mesh(output_pointcloud,faces)
    
    

In [97]:
visualize_random()

(185, 2)
GT Mesh


Output()

Predicted Pointcloud


Output()

(156, 3)
(48, 2)
156


Output()

In [26]:
def export_mesh_to_obj(path, vertices, faces):

    to_write = ""

    faces += 1

    obj_file = open(path, "w+")

    for i in range(vertices.shape[0]):
        vx = vertices[i,:]
        l = f"v {vx[0]} {vx[1]} {vx[2]}\n"
        to_write += l

    for i in range(faces.shape[0]):
        face = faces[i,:]
        l = f"f {face[0]} {face[1]} {face[2]}\n"
        to_write += l

    obj_file.write(to_write)

    obj_file.close()


In [27]:
import open3d

In [28]:
mesh_in = open3d.io.read_triangle_mesh("output.obj")

In [29]:
mesh_in

TriangleMesh with 100 points and 35181 triangles.

In [30]:
mesh_smp = mesh_in.simplify_quadric_decimation(target_number_of_triangles=20000)

In [31]:
mesh_smp

TriangleMesh with 31 points and 19626 triangles.

In [32]:
open3d.visualization.draw_geometries([mesh_smp])

In [33]:
open3d.io.write_triangle_mesh("simplified.obj", mesh_smp)

True

In [35]:
np.array(mesh_smp.vertices)

array([[ 0.1610322 , -0.29845652, -0.28329909],
       [-0.14093108, -0.02186579, -0.24869767],
       [ 0.00977515,  0.0844263 , -0.2368125 ],
       [ 0.16424899, -0.00437306, -0.23929292],
       [-0.15810894, -0.33066337, -0.24130594],
       [ 0.07457884, -0.03870672, -0.19639743],
       [ 0.14389685, -0.27245843, -0.12635003],
       [ 0.13066384, -0.3826485 , -0.22867607],
       [ 0.14924926,  0.13215493, -0.21904702],
       [ 0.15126379, -0.36364358, -0.07179864],
       [ 0.14431539, -0.09431536,  0.08698647],
       [ 0.14917452,  0.08565598,  0.22025901],
       [ 0.15639293, -0.27986974,  0.20635703],
       [ 0.16032607,  0.10863305,  0.11851151],
       [ 0.13281418,  0.07716768,  0.00740832],
       [ 0.14441749, -0.01945177,  0.16263057],
       [-0.03637091, -0.04049452,  0.11493398],
       [ 0.13897389,  0.42027318,  0.14827853],
       [-0.10913445,  0.11843286,  0.15846808],
       [ 0.03900905,  0.47646358,  0.19953372],
       [-0.1440687 , -0.19119753,  0.186

In [39]:
np.array(mesh_smp.triangles)

array([[ 0,  1,  2],
       [ 0,  1,  3],
       [ 0,  1,  4],
       ...,
       [25, 18, 16],
       [25, 18, 20],
       [25, 18, 20]], dtype=int32)